In [3]:
import os 
os.getcwd()

'C:\\kiosks_'

In [2]:
os.chdir('C:/kiosks_')

In [4]:
from main.functions.common_function import read_yaml_file, create_directories
from pathlib import Path
class speech_recog_config:
    def __init__(self):
        self.config = read_yaml_file(Path('config/config.yaml'))
        self.params = read_yaml_file(Path('config/params.yaml'))

    def speech_recog_module(self):
        config = self.config.local_models
        create_directories([config.speech_recog_model])

        sppech_recog_config = {
            'model_path': Path(config['speech_recog_model']),
            'model': config['speech_to_text_model'],
            'duration': self.params['duration'],
            'sample_rate': self.params['sampling_rate'],
            'channel': self.params['channel'],
        }
        
        return sppech_recog_config
        


In [26]:

from faster_whisper import WhisperModel
import torch
import sounddevice
from scipy.io.wavfile import write


class speech_recog_compo:
    def __init__(self, config, parmas):
        # obj = speech_recog()
        # self.configs = obj.speech_recog_module()
        # self.model_path = self.configs['model_path']
        #  print(self.configs)
        self.config = config
        self.params = parmas


    def download_model(self):
        os.environ["HF_HUB_DISABLE_SYMLINKS"] = "1"  # <-- ADD THIS LINE
        if torch.cuda.is_available():
            device = "cuda"
        else:
            device = "cpu"
        
        model_size = self.config['model']
        model = WhisperModel(model_size, download_root= self.config['model_path'], device=device, compute_type="float32")
        print('model succefully downloaded', model)

    
    def record_and_transcribe_audio(self):
        print('Recording audio...')
        audio = sounddevice.rec(int(self.params['duration'] * self.params['sample_rate']), samplerate=self.params['sample_rate'], channels=1, dtype='int16')
        sounddevice.wait()
        write('recording.wav', self.params['sample_rate'], audio)
        print("✅ Recording saved to", "recording.wav")
        model = WhisperModel(self.config['model'], compute_type="float32")  # use float16 if GPU available

        segments, info = model.transcribe("recording.wav", beam_size=5)
        print("🌐 Detected Language:", info.language)
        print("📝 Transcription:")
        for segment in segments:
            print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
    

In [27]:
obj = speech_recog_config()
aa = obj.speech_recog_module()

created directory at: models/speech_recog_model


In [28]:

f = speech_recog_compo(aa, aa)
f.download_model()
f.record_and_transcribe_audio()


model succefully downloaded <faster_whisper.transcribe.WhisperModel object at 0x0000026427429310>
Recording audio...
✅ Recording saved to recording.wav


c:\kiosks_\myenv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--Systran--faster-whisper-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


🌐 Detected Language: en
📝 Transcription:


In [29]:
import sounddevice as sd
from scipy.io.wavfile import write

def record_audio(filename="recording.wav", duration=5, samplerate=16000):
    print("🎤 Recording...")
    audio = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()
    write(filename, samplerate, audio)
    print("✅ Recording saved to", filename)


In [ ]:
from faster_whisper import WhisperModel

def transcribe_audio(filename="recording.wav"):
    model = WhisperModel("base", compute_type="float32")  # use float16 if GPU available
    # model = WhisperModel("large-v3", compute_type="float32")  # use float16 if GPU available

    segments, info = model.transcribe(filename, beam_size=5)
    print("🌐 Detected Language:", info.language)
    print("📝 Transcription:")
    for segment in segments:
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")




if __name__ == "__main__":
    record_audio(duration=8)  # speak something in any language
    transcribe_audio("recording.wav")



🎤 Recording...
✅ Recording saved to recording.wav
🌐 Detected Language: en
📝 Transcription:
[0.00s -> 4.00s]  Hello, hello, hello


In [24]:
import openai
from dotenv import load_dotenv
import os
import sounddevice
from scipy.io.wavfile import write
load_dotenv()
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))  # new client instance


In [ ]:
def record_audio(filename="recording_api_whisper.wav", duration=5, samplerate=16000):
    print("🎤 Recording...")
    audio = sounddevice.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sounddevice.wait()
    write(filename, samplerate, audio)
    print("✅ Recording saved to", filename)


def transcribe_with_openai(filename="recording_api_whisper.wav"):
    with open(filename, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text"
        )
    print("📝 Transcription:", transcript)


In [26]:
if __name__ == "__main__":
    record_audio(duration=5)  # from earlier
    transcribe_with_openai("recording_api_whisper.wav")


🎤 Recording...
✅ Recording saved to recording_api_whisper.wav
📝 Transcription: 



In [3]:
import os
os.chdir('C:/kiosks_')
os.getcwd()

'C:\\kiosks_'

In [7]:
from main.components.speech_recognition.config.speech_recog_config import speech_recog_config
from faster_whisper import WhisperModel
import torch
import speech_recognition as sr
from scipy.io.wavfile import write
import os, wave, io

In [ ]:

class speech_recog_compo:
    def __init__(self, config, parmas):
        # obj = speech_recog()
        # self.configs = obj.speech_recog_module()
        # self.model_path = self.configs['model_path']
        #  print(self.configs)
        self.config = config
        self.params = parmas
        self.is_recording = False



    def download_model(self):
        os.environ["HF_HUB_DISABLE_SYMLINKS"] = "1"  # <-- ADD THIS LINE
        if torch.cuda.is_available():
            device = "cuda"
        else:
            device = "cpu"
        
        model_size = self.config['model']
        model = WhisperModel(model_size, download_root= self.config['model_path'], device=device, compute_type="float32")
        print('model succefully downloaded', model)

    
    def record_and_transcribe_audio(self):
        print("🎙️ Listening... Speak now. Recording will stop automatically when you're silent.\n(Press Ctrl+C to stop manually)\n")

        recog = sr.Recognizer()
        mic = sr.Microphone(sample_rate=self.params['sample_rate'])
        all_audio = []

        # try:
        with mic as source:
            recog.adjust_for_ambient_noise(source)
            while True:
                print("🎧 Listening for next phrase...")
                try:
                    audio = recog.listen(source, timeout=2, phrase_time_limit=10)
                    all_audio.append(audio.get_wav_data())
                    print("✅ Phrase captured.")
                except sr.WaitTimeoutxError:
                    print("⏹️ Detected silence. Ending recording.")
                    break
        # except KeyboardInterrupt:
        #     print("\n🛑 Manual stop triggered.")

        if not all_audio:
            print("⚠️ No audio was recorded.")
            return

        wav_file = "recording.wav"
        self._combine_audio_data(all_audio, wav_file)
        print("✅ Recording saved to:", wav_file)

        self._transcribe_audio(wav_file)
    
    def _combine_audio_data(self, audio_data_chunks, output_filename):
        """Combine multiple audio chunks into a single WAV file"""
        if not audio_data_chunks:
            return
        
        # Get parameters from the first chunk
        with wave.open(io.BytesIO(audio_data_chunks[0]), 'rb') as first_chunk:
            params = first_chunk.getparams()
        
        # Write all chunks to a single file
        with wave.open(output_filename, 'wb') as output_file:
            output_file.setparams(params)

            for chunk in audio_data_chunks:
                with wave.open(io.BytesIO(chunk), 'rb') as chunk_file:
                    output_file.writeframes(chunk_file.readframes(chunk_file.getnframes()))
    
    def _transcribe_audio(self, wav_file):
        """Transcribe the recorded audio using Whisper"""
        model = WhisperModel(self.config['model'], compute_type=self.config['compute_type'])
        
        segments, info = model.transcribe(wav_file, log_progress=True, beam_size=5)
        print("🌐 Detected Language:", info.language)
        print("📝 Transcription:")
        for segment in segments:
            print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")



In [18]:

obj = speech_recog_config()
config_params = obj.speech_recog_module()

f = speech_recog_compo(config_params, config_params)
f.download_model()
f.record_and_transcribe_audio()

created directory at: models/speech_recog_model
model succefully downloaded <faster_whisper.transcribe.WhisperModel object at 0x000002AB90542720>
🎙️ Listening... Speak now. Recording will stop automatically when you're silent.
(Press Ctrl+C to stop manually)

🎧 Listening for next phrase...
✅ Phrase captured.
🎧 Listening for next phrase...
✅ Phrase captured.
🎧 Listening for next phrase...
✅ Phrase captured.
🎧 Listening for next phrase...
✅ Phrase captured.
🎧 Listening for next phrase...
✅ Phrase captured.
🎧 Listening for next phrase...
⏹️ Detected silence. Ending recording.
✅ Recording saved to: recording.wav
🌐 Detected Language: en
📝 Transcription:


100%|██████████| 15.55/15.55 [00:03<00:00,  3.99seconds/s]

[0.00s -> 6.00s]  I hope this isn't continuously speaking. Until I stop speaking, it will just...
[6.00s -> 12.00s]  Until I just stop speaking, it will continue. Until I just keep talking, it will keep listening.
[12.00s -> 16.00s]  Just that's it. Now I'm gonna hop. So it will be like...
